# Extra Trees Classifiers
---

## Introduction

The third models we are going to use to try tackling the current problem are **extra trees classifiers**.

As always, the prerequisite step consists on loading the appropriate packages to perform our work:

In [1]:
# Activate 'airbnb' environment:
!source activate airbnb

In [2]:
# Needed packages:
import numpy as np
import pandas as pd
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.externals import joblib
from utils import (create_training_testing_datasets,
                   calculate_dcg,
                   calculate_ndcg,
                   clf_prediction,
                   ndcg_mean_score_calc,
                   detailed_ndcg_mean_score_calc)

/Applications/anaconda3/envs/airbnb/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


---

## Create training and testing datasets

In [3]:
# Load the data:
consolidated_dataset = pd.read_csv("../data/consolidated_dataset.csv")

# Check basic info:
print("*** Some basic info:")
print("'consolidated_dataset' has {} data points with {} variables each.".format(*consolidated_dataset.shape))
print("'consolidated_dataset' counts {} missing values.".format(consolidated_dataset.isnull().sum().sum()))

# Give a look to the first lines:
print("\n*** First lines:")
display(consolidated_dataset.head())

*** Some basic info:
'consolidated_dataset' has 213451 data points with 161 variables each.
'consolidated_dataset' counts 0 missing values.

*** First lines:


,age,country_destination,nans,day_account_created,weekday_account_created,week_account_created,month_account_created,year_account_created,day_first_active,weekday_first_active,...,first_browser_SeaMonkey,first_browser_Silk,first_browser_SiteKiosk,first_browser_SlimBrowser,first_browser_Sogou Explorer,first_browser_Stainless,first_browser_TenFourFox,first_browser_TheWorld Browser,first_browser_Yandex.Browser,first_browser_wOSBrowser
0,-1.0,NDF,1.225078,28,0,26,6,2010,19,3,...,0,0,0,0,0,0,0,0,0,0
1,38.0,NDF,-0.453135,25,2,21,5,2011,23,5,...,0,0,0,0,0,0,0,0,0,0
2,56.0,US,-0.453135,28,1,39,9,2010,9,1,...,0,0,0,0,0,0,0,0,0,0
3,42.0,other,-0.453135,5,0,49,12,2011,31,5,...,0,0,0,0,0,0,0,0,0,0
4,41.0,US,0.385972,14,1,37,9,2010,8,1,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# Create training and testing datasets:
X_train, X_test, y_train, y_test, encoding_dict = create_training_testing_datasets(consolidated_dataset)

---

## Calculate Normalized DCG scores

### "Out-of-the-box" extra trees classifier

In [5]:
# Calculate weight of each class:
dest_weight = {x: 0 for x in range(12)}
for country_dest in y_train:
    dest_weight[country_dest] += 1

# Initialize the classifier:
et_clf = ExtraTreesClassifier(random_state=42, class_weight=dest_weight)

# Train the classifier:
print("Time info about classifier training:")
%time ootb_et_clf = et_clf.fit(X_train, y_train)

Time info about classifier training:


/Applications/anaconda3/envs/airbnb/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


CPU times: user 9.49 s, sys: 297 ms, total: 9.78 s
Wall time: 10.1 s


In [6]:
# Perform one prediction to check classifier:
print("Time info about classifier prediction:")
%time preds_list = clf_prediction(ootb_et_clf, X_train[0])
print("***")

# Reverse encoding dictionary:
decoding_dict = dict(map(reversed, encoding_dict.items()))

# Print result:
print("For the classifier check:")
print("- Real first booking destination country: {}".format(decoding_dict[y_train[0]]))
print("- Predictions list: {}".format([decoding_dict[x] for x in preds_list]))

Time info about classifier prediction:
CPU times: user 5.92 ms, sys: 9.95 ms, total: 15.9 ms
Wall time: 650 ms
***
For the classifier check:
- Real first booking destination country: US
- Predictions list: ['US']


In [7]:
# Calculate nDCG mean score on training dataset:
print("Time info about nDCG mean score calculation on training dataset:")
%time ndcg_mean_score = ndcg_mean_score_calc(ootb_et_clf, X_train, y_train)
print("***")
print("On training dataset, classifier nDCG mean score is {:.6f}.".format(ndcg_mean_score))

Time info about nDCG mean score calculation on training dataset:
CPU times: user 3min 5s, sys: 1.61 s, total: 3min 7s
Wall time: 3min 6s
***
On training dataset, classifier nDCG mean score is 1.000000.


In [8]:
# Calculate nDCG mean score on testing dataset:
print("Time info about nDCG mean score calculation on testing dataset:")
%time ndcg_mean_score = ndcg_mean_score_calc(ootb_et_clf, X_test, y_test)
print("***")
print("On testing dataset, classifier nDCG mean score is {:.6f}.".format(ndcg_mean_score))

Time info about nDCG mean score calculation on testing dataset:
CPU times: user 47.2 s, sys: 411 ms, total: 47.6 s
Wall time: 47.3 s
***
On testing dataset, classifier nDCG mean score is 0.725702.


In [9]:
# Calculate nDCG mean score for each class on testing dataset:
print("Time info about nDCG mean score calculation for each class on testing dataset:")
%time ndcg_mean_scores_list = detailed_ndcg_mean_score_calc(ootb_et_clf, X_test, y_test, encoding_dict)
print("***")
print("Detailed results for each class on testing dataset:")
for country_dest in range(12):
    print("nDCG mean score for {}: {:.6f}".format(decoding_dict[country_dest],
                                                  ndcg_mean_scores_list[country_dest]))

Time info about nDCG mean score calculation for each class on testing dataset:
CPU times: user 47.3 s, sys: 498 ms, total: 47.8 s
Wall time: 47.5 s
***
Detailed results for each class on testing dataset:
nDCG mean score for AU: 0.018683
nDCG mean score for CA: 0.024892
nDCG mean score for DE: 0.028022
nDCG mean score for ES: 0.040555
nDCG mean score for FR: 0.092243
nDCG mean score for GB: 0.047233
nDCG mean score for IT: 0.056431
nDCG mean score for NDF: 0.868335
nDCG mean score for NL: 0.015286
nDCG mean score for PT: 0.010016
nDCG mean score for US: 0.702779
nDCG mean score for other: 0.198947


In [10]:
# Save model:
joblib.dump(ootb_et_clf, "../models/ootb_et_clf.pkl")

['../models/ootb_et_clf.pkl']

### "Optimized" extra trees classifier

In [11]:
# Set parameters of the random grid:
n_estimators = [int(x) for x in np.linspace(10, 100, num=5)]
max_depth = [int(x) for x in np.linspace(10, 100, num=5)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
max_features = ['auto', 'sqrt']
bootstrap = [True, False]
random_state = [42]
class_weight = []
class_weight.append(dest_weight)

# Create the random grid:
random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'max_features': max_features,
               'bootstrap': bootstrap,
               'random_state': random_state,
               'class_weight': class_weight}

# Perform randomized search cross validation:

et_clf = ExtraTreesClassifier()

et_clf_random = RandomizedSearchCV(estimator=et_clf,
                                   param_distributions=random_grid,
                                   n_iter=20,
                                   n_jobs=1,
                                   random_state=42,
                                   verbose=0)

print("Time info about randomized search cross validation:")
%time et_clf_fit = et_clf_random.fit(X_train, y_train)

# Get the "best" classifier:
opt_et_clf = et_clf_fit.best_estimator_

Time info about randomized search cross validation:


/Applications/anaconda3/envs/airbnb/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


CPU times: user 24min 4s, sys: 48.3 s, total: 24min 52s
Wall time: 23min 22s


In [12]:
# Parameters of the "best" classifier:
print("*** Parameters of the 'best' classifier:")
for param, param_value in et_clf_fit.best_params_.items():
    print("- Paramater {}: {}".format(param, param_value))

*** Parameters of the 'best' classifier:
- Paramater random_state: 42
- Paramater n_estimators: 55
- Paramater min_samples_split: 10
- Paramater min_samples_leaf: 1
- Paramater max_features: auto
- Paramater max_depth: 100
- Paramater class_weight: {0: 431, 1: 1142, 2: 849, 3: 1799, 4: 4018, 5: 1859, 6: 2268, 7: 99634, 8: 610, 9: 174, 10: 49901, 11: 8075}
- Paramater bootstrap: False


In [13]:
# Perform one prediction to check classifier:
print("Time info about classifier prediction:")
%time preds_list = clf_prediction(opt_et_clf, X_train[0])
print("***")

# Reverse encoding dictionary:
decoding_dict = dict(map(reversed, encoding_dict.items()))

# Print result:
print("For the classifier check:")
print("- Real first booking destination country: {}".format(decoding_dict[y_train[0]]))
print("- Predictions list: {}".format([decoding_dict[x] for x in preds_list]))

Time info about classifier prediction:
CPU times: user 9.8 ms, sys: 3.04 ms, total: 12.8 ms
Wall time: 11 ms
***
For the classifier check:
- Real first booking destination country: US
- Predictions list: ['US', 'NDF', 'other', 'FR', 'IT', 'CA', 'GB', 'NL', 'DE']


In [14]:
# Calculate nDCG mean score on training dataset:
print("Time info about nDCG mean score calculation on training dataset:")
%time ndcg_mean_score = ndcg_mean_score_calc(opt_et_clf, X_train, y_train)
print("***")
print("On training dataset, classifier nDCG mean score is {:.6f}.".format(ndcg_mean_score))

Time info about nDCG mean score calculation on training dataset:
CPU times: user 13min 55s, sys: 8.19 s, total: 14min 3s
Wall time: 13min 59s
***
On training dataset, classifier nDCG mean score is 0.882097.


In [15]:
# Calculate nDCG mean score on testing dataset:
print("Time info about nDCG mean score calculation on testing dataset:")
%time ndcg_mean_score = ndcg_mean_score_calc(opt_et_clf, X_test, y_test)
print("***")
print("On testing dataset, classifier nDCG mean score is {:.6f}.".format(ndcg_mean_score))

Time info about nDCG mean score calculation on testing dataset:
CPU times: user 3min 25s, sys: 643 ms, total: 3min 26s
Wall time: 3min 26s
***
On testing dataset, classifier nDCG mean score is 0.813936.


In [16]:
# Calculate nDCG mean score for each class on testing dataset:
print("Time info about nDCG mean score calculation for each class on testing dataset:")
%time ndcg_mean_scores_list = detailed_ndcg_mean_score_calc(opt_et_clf, X_test, y_test, encoding_dict)
print("***")
print("Detailed results for each class on testing dataset:")
for country_dest in range(12):
    print("nDCG mean score for {}: {:.6f}".format(decoding_dict[country_dest],
                                                  ndcg_mean_scores_list[country_dest]))

Time info about nDCG mean score calculation for each class on testing dataset:
CPU times: user 3min 27s, sys: 2.01 s, total: 3min 29s
Wall time: 3min 27s
***
Detailed results for each class on testing dataset:
nDCG mean score for AU: 0.007570
nDCG mean score for CA: 0.040835
nDCG mean score for DE: 0.025496
nDCG mean score for ES: 0.133674
nDCG mean score for FR: 0.350670
nDCG mean score for GB: 0.111953
nDCG mean score for IT: 0.165946
nDCG mean score for NDF: 0.975789
nDCG mean score for NL: 0.013014
nDCG mean score for PT: 0.000000
nDCG mean score for US: 0.715075
nDCG mean score for other: 0.466886


In [17]:
# Save model:
joblib.dump(opt_et_clf, "../models/opt_et_clf.pkl")

['../models/opt_et_clf.pkl']

---

## Conclusion

As a quick conclusion, for the results obtained for the last "optimized" extra trees classifier, we can note 4 major elements:
* On testing dataset, it gets a better nDCG mean score than the one obtained by the naive model.
* On testing dataset, it gets better nDCG mean scores for predicting correctly Australia, Canada, Germany, Spain, Great Britain, Netherlands and USA than the ones obtained by the naive model.
* On testing dataset, it gets worse nDCG mean scores for predicting correctly France, Italy, no destination found and other than the ones obtained by the naive model.
* On testing dataset, it is as "bad" as the naive model for predicting correctly Portugal.